In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [2]:
train_dir = '1train'
test_dir = '1test'

In [4]:
input_shape = (48, 48, 3)
batch_size = 32
epochs = 80
num_classes = 7

In [4]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

In [5]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(48, 48),
    color_mode='rgb',
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)

Found 5830 images belonging to 7 classes.


In [6]:
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(48, 48),
    color_mode='rgb',
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

Found 1467 images belonging to 7 classes.


In [7]:
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=input_shape)

In [8]:
for layer in base_model.layers:
    layer.trainable = False

In [9]:
model = Sequential([
    base_model,
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

In [10]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [11]:
checkpoint = ModelCheckpoint('efficientnet_emotion_model.keras', monitor='val_accuracy', save_best_only=True, mode='max')
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [12]:
history = model.fit(
    train_generator,
    validation_data=test_generator,
    epochs=epochs,
    callbacks=[checkpoint, early_stopping]
)

Epoch 1/5


C:\ProgramData\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


183/183 ━━━━━━━━━━━━━━━━━━━━ 163s 650ms/step - accuracy: 0.1966 - loss: 2.0160 - val_accuracy: 0.2461 - val_loss: 1.8278
Epoch 2/5
183/183 ━━━━━━━━━━━━━━━━━━━━ 30s 158ms/step - accuracy: 0.2337 - loss: 1.8517 - val_accuracy: 0.2461 - val_loss: 1.8276
Epoch 3/5
183/183 ━━━━━━━━━━━━━━━━━━━━ 31s 163ms/step - accuracy: 0.2514 - loss: 1.8379 - val_accuracy: 0.2461 - val_loss: 1.8194
Epoch 4/5
183/183 ━━━━━━━━━━━━━━━━━━━━ 34s 180ms/step - accuracy: 0.2465 - loss: 1.8319 - val_accuracy: 0.2461 - val_loss: 1.8237
Epoch 5/5
183/183 ━━━━━━━━━━━━━━━━━━━━ 36s 191ms/step - accuracy: 0.2507 - loss: 1.8369 - val_accuracy: 0.2461 - val_loss: 1.8165


In [13]:
model.save('final_efficientnet_emotion_model.keras')